# This notebook will be used to plot VADs relative to the storm

In [ ]:
import pyart
import numpy as np
from datetime import datetime, timedelta
import glob
# from radarcalc import *
import matplotlib.pyplot as plt
import pandas as pd
import metpy.calc as mpcalc
import metpy
import metpy.plots
from metpy.units import units
import cartopy.crs as ccrs
import gc
from astropy.convolution import convolve
from boto.s3.connection import S3Connection
import tempfile
import copy
import matplotlib
import xarray as xr
import math
from datetime import datetime

In [ ]:
def find_nearest(array, value):
    '''
    Function to find index of the array in which the value is closest to

    Parameters: array (array), value (number)
    Returns: index (int)

    Example: xind = CM1calc.find_nearest(x,5)
    '''

    array = np.asarray(array)
    idx = (np.abs(array-value)).argmin()
    return idx


def dealias_Ka(radar,PPIGC_flag=False):
    '''
    This function aims to take care of all the nitty gritty customizations of pyarts dealiasing specifically for TTUKa radar data.

    Parameter: radar (pyart object), PPIGC_flag (boolean *kwarg)
    Returns: radar (pyart object) with corrected_velocity field and velocity_texture field added on

    How it works:
    1. Calculates velocity texture and creates a histogram based on all of the magnitudes of textures at each bin for the entire volume
    2. loops through each indivual sweep
    3. infinite loop to iterate between the minimum amount of texture between textures 1 and 6 and the maximum amount of texture in the histogram, with 0.5 m/s steps
    4. Each iteration, create a gatefilter that filters the bins above that texture value and dealias it with the gatefilter to ignore the high textured regions
    5. If the maximum texture is reached before breaking, set the gatefilter to mask textures above 12 m/s
    6. if the scan is an RHI, run a 2 pass variance filter along each ray.
        This convolves a 71 sized boxcar with the data, and if the difference between the point the boxcar is centered on and the mean of the boxcar is greater than the nyquist, then add/subtract 2*nyquist to that point
    7. Then, take the difference between the mean of the bottom 4 rays of the dealiased velocity in the RHI and the bottom 4 rays of the aliased velocity
    8. If the absolute difference is larger than the nyquist, the either subtract 2* nyquist or add 2*nyquist to the entire sweep and break the loop
    9. If the absolute difference is less than the nyquist, no fixes need to be applied, break the loop
    10. If the scan is a PPI, do steps 8 and 9, but skip steps 6 and 7
    11. Outside the infinite loop, add the option of the PPIGC_flag, a boolean flag that will help maintain ground clutter in PPIs at 0 m/s
        This works by identifying regions of very low spectrum width (<0.1 m/s), and setting the velocity in those regions = 0. Please note, this may introduce artificial speckles of 0 in real data, where spectrum width is noisy
    12. Apply the alias fix algorithm which convolves a boxcar of specified boxcar of size 9 with the data, and if the variance between the middle pixel of interest and the mean is greater than the nyquist, then flip it back over
    12. At the end of each sweep, assign the data from that processed sweep into a dictionary, then add the dictionary as the corrected_velocity field.
    
    The aforementioned method is FAR from perfect, but is as robust as I can do currently. One thing to improve this though is to use the UNRAVEL algorithm: https://github.com/vlouf/dealias
    The UNRAVEL algorithm shows remarkable error characteristics compared to "competitors", possibly at a time cost, which isn't a HUGE deal for us. Downside is it may not work for our "volumes" since they are temporally uncorrelated and are not full volumes through the atmosphere
    '''
    
    vel_texture = pyart.retrieve.calculate_velocity_texture(radar, vel_field='velocity', wind_size=3)
    radar.add_field('velocity_texture', vel_texture, replace_existing=True)
    hist, bins = np.histogram(radar.fields['velocity_texture']['data'][~np.isnan(radar.fields['velocity_texture']['data'])], bins=150)
    bins = (bins[1:]+bins[:-1])/2.0
    gatefilter = pyart.filters.GateFilter(radar)
    velocity_dealiased = pyart.correct.dealias_region_based(radar, vel_field='velocity', nyquist_vel=radar.instrument_parameters['nyquist_velocity']['data'][0], centered=True) #standin, data will be replaced

    for swp_id in range(radar.nsweeps):
        #get indices from beginning and ending of sweep
        sw_start = radar.sweep_start_ray_index['data'][swp_id]
        sw_end = radar.sweep_end_ray_index['data'][swp_id]+1

        counter = 0
        while True: #do an infinite loop and either break it when the data is unfolded correctly or when the max texture is reached
            #if the bin with the lowest count between textures 1 and 6 + i*0.5 is less than the maximum amount of bins
            if bins[np.where(hist==np.min(hist[find_nearest(bins,1):find_nearest(bins,6)]))[0][0]]+counter*0.5 < np.amax(bins):
                gatefilter.exclude_above('velocity_texture', bins[np.where(hist==np.min(hist[find_nearest(bins,1):find_nearest(bins,6)]))[0][0]]+counter*0.5)
                nyq = radar.instrument_parameters['nyquist_velocity']['data'][0]
                vede = pyart.correct.dealias_region_based(radar, vel_field='velocity', nyquist_vel=nyq,
                                                                        centered=True, gatefilter=gatefilter)
            else:
                gatefilter.exclude_above('velocity_texture', 12)#bins[np.where(hist==np.min(hist[find_nearest(bins,1):find_nearest(bins,6)]))[0][0]])
                nyq = radar.instrument_parameters['nyquist_velocity']['data'][0]
                vede = pyart.correct.dealias_region_based(radar, vel_field='velocity', nyquist_vel=nyq,
                                                                        centered=True, gatefilter=gatefilter)


            np.ma.set_fill_value(vede['data'], np.nan)
            #extract mask so we can apply the correct gatefilters on later
            mask=np.ma.getmask(vede['data'])

            #apply mask to velocity field and fix the small blips from dealiasing
            if radar.scan_type == 'rhi':
                #pass 1 of variance filtering along the ray.
                #Convolves a 71 sized boxcar with the data, and if the difference between the point the boxcar is centered on and the mean of the boxcar is greater than the nyquist, then add/subtract 2*nyquist to that point
                vel = vede['data'].filled()

                for sw in range(np.shape(vel)[0]):
                    mean = convolve(vel[sw,:],np.ones(71))
                    var = vel[sw,:]-mean
                    high_idx = var > nyq
                    low_idx = var < -nyq
                    vel[sw,:][high_idx] = vel[sw,:][high_idx] - 2*nyq
                    vel[sw,:][low_idx] = vel[sw,:][low_idx] + 2*nyq
                vede['data']=np.ma.masked_array(vel,mask=mask,fill_value=np.nan)

                #pass 2 of variance filtering along the ray. In case there are errant folds than need to be folded back
                vel = vede['data'].filled()
                for sw in range(np.shape(vel)[0]):
                    mean = convolve(vel[sw,:],np.ones(71))
                    var = vel[sw,:]-mean
                    high_idx = var > nyq
                    low_idx = var < -nyq
                    vel[sw,:][high_idx] = vel[sw,:][high_idx] - 2*nyq
                    vel[sw,:][low_idx] = vel[sw,:][low_idx] + 2*nyq
                vede['data']=np.ma.masked_array(vel,mask=mask,fill_value=np.nan)

                #find means of the bottom 4 rays of the RHI(should be close to 0) and compare the dealiased velocities to the aliased velocities
                np.ma.set_fill_value(radar.fields['velocity']['data'], np.nan)
                meanvelal = np.mean(radar.fields['velocity']['data'][sw_start:sw_start+4,:].filled()[~np.isnan(radar.fields['velocity']['data'][sw_start:sw_start+4,:].filled())])
                meanveldeal = np.mean(vede['data'][sw_start:sw_start+4,:].filled()[~np.isnan(vede['data'][sw_start:sw_start+4,:].filled())])
                if np.abs(meanvelal-meanveldeal) < nyq: #nyq is an arbitrary threshold and should be tuned
                    break
                if bins[np.where(hist==np.min(hist[find_nearest(bins,1):find_nearest(bins,6)]))[0][0]]+counter*0.5 < np.amax(bins):
                    if (meanvelal-meanveldeal) > 0:
                        vede['data'][sw_start:sw_end,:] += 2*nyq
                    else:
                        vede['data'][sw_start:sw_end,:] -= 2*nyq
                    break
            if radar.scan_type == 'ppi':
                np.ma.set_fill_value(radar.fields['velocity']['data'], np.nan)
                meanvelal = np.mean(radar.fields['velocity']['data'][sw_start:sw_end,:].filled()[~np.isnan(radar.fields['velocity']['data'][sw_start:sw_end,:].filled())])
                meanveldeal = np.mean(vede['data'][sw_start:sw_end,:].filled()[~np.isnan(vede['data'][sw_start:sw_end,:].filled())])
                if np.abs(meanvelal-meanveldeal) < nyq: #nyq is an arbitrary threshold and should be tuned
                    break
                if bins[np.where(hist==np.min(hist[find_nearest(bins,1):find_nearest(bins,6)]))[0][0]]+counter*0.5 < np.amax(bins):
                    if (meanvelal-meanveldeal) > 0:
                        vede['data'][sw_start:sw_end,:] += 2*nyq
                    else:
                        vede['data'][sw_start:sw_end,:] -= 2*nyq
                    break
            counter+=1
            
        #put alias fix inside here instead of calling it to make it more portable
        delta=3
        mean = convolve(vede['data'][sw_start:sw_end,:],np.ones((delta,delta))/delta**2.)
        mean[0,:] = vede['data'][sw_start:sw_end,:][0,:]
        mean[-1,:] = vede['data'][sw_start:sw_end,:][-1,:]
        var = vede['data'][sw_start:sw_end,:] - mean

        high_idx = np.logical_and(var > nyq, var < 4*nyq)
        low_idx = np.logical_and(var < -nyq, var > -4*nyq)

        vede['data'][sw_start:sw_end,:][high_idx] = vede['data'][sw_start:sw_end,:][high_idx] - 2*nyq
        vede['data'][sw_start:sw_end,:][low_idx] = vede['data'][sw_start:sw_end,:][low_idx] + 2*nyq

        #corrects ground clutter by arbitrarily setting the velocity equal to 0 where spectrum width is less than 0.075 m/s
        if PPIGC_flag == True:
            if radar.scan_type == 'ppi':
                sw = radar.fields['spectrum_width']['data'][sw_start:sw_end,:].filled()
                vel = radar.fields['velocity']['data'][sw_start:sw_end,:].filled()
                mask = sw<0.1
                vede['data'][sw_start:sw_end,:] = np.where(~mask,vede['data'][sw_start:sw_end,:],0)

        velocity_dealiased['data'][sw_start:sw_end,:] = vede['data'][sw_start:sw_end,:]
        velocity_dealiased['data'][sw_start:sw_end,:] = alias_fix(velocity_dealiased['data'][sw_start:sw_end,:],nyq,delta=9)
    radar.add_field('corrected_velocity', velocity_dealiased, replace_existing=True)

    return radar

def alias_fix(vel,nyq,delta=3):
    '''
    !!!!!!!!!!!!!!!!!!
    Removes dealiasing errors around the periphery of a folded region

    Parameters: velocity array (array), nyquist velocity (number), size of window (int, must be odd, unity is no change)
    Returns: cleaned velocity array (array)
    '''
    mean = convolve(vel,np.ones((delta,delta))/delta**2.)
    mean[0,:] = vel[0,:]
    mean[-1,:] = vel[-1,:]
    var = vel - mean

    high_idx = np.logical_and(var > nyq, var < 4*nyq)
    low_idx = np.logical_and(var < -nyq, var > -4*nyq)

    vel[high_idx] = vel[high_idx] - 2*nyq
    vel[low_idx] = vel[low_idx] + 2*nyq

    return vel

def get_radar_from_aws_sam(site, datetime_t, datetime_te):
    """
    Get the closest volume of NEXRAD data to a particular datetime.
    Parameters
    ----------
    site : string
        four letter radar designation
    datetime_t : datetime
        desired date time
    Returns
    -------
    radar : Py-ART Radar Object
        Radar closest to the queried datetime
    """
    # SAM: added nexradaws to scan bucket instead of boto because boto was causing weird timeout errors...
    import nexradaws
    import os
    
    conn = nexradaws.NexradAwsInterface()

    scans = conn.get_avail_scans_in_range(datetime_t-timedelta(minutes=15), datetime_te+timedelta(minutes=15), site)

    scans = [scan for scan in scans if "_MDM" not in scan.filename]

    keys = [scan.filename for scan in scans]
    datetimes = [scan.scan_time.replace(tzinfo=None) for scan in scans]
    
    # This is alex's code again:
    # find the closest available radar to your datetime
    closest_datetime_b = _nearestDate(datetimes, datetime_t)
    closest_datetime_e = _nearestDate(datetimes, datetime_te)

    index_b = datetimes.index(closest_datetime_b)
    index_e = datetimes.index(closest_datetime_e)

    radar_namelist = keys[index_b:index_e+1]

    # SAM: slightly modified download routine
    remote_scan_list = scans[index_b:index_e+1]
    radar_list = []
    for remote_scan in remote_scan_list:
        conn.download(remote_scan, '.')
        rdr = pyart.io.read(remote_scan.filename)
        radar_list.append(rdr)
        os.remove(remote_scan.filename)
    return radar_namelist,radar_list

    



def get_radar_from_aws(site, datetime_t, datetime_te):
    """
    Get the closest volume of NEXRAD data to a particular datetime.
    Parameters
    ----------
    site : string
        four letter radar designation
    datetime_t : datetime
        desired date time
    Returns
    -------
    radar : Py-ART Radar Object
        Radar closest to the queried datetime
    """

    # First create the query string for the bucket knowing
    # how NOAA and AWS store the data
    my_pref = datetime_t.strftime('%Y/%m/%d/') + site

    # Connect to the bucket
    conn = S3Connection(anon = True)
    bucket = conn.get_bucket('noaa-nexrad-level2')

    # Get a list of files
    bucket_list = list(bucket.list(prefix = my_pref))

    # we are going to create a list of keys and datetimes to allow easy searching
    keys = []
    datetimes = []

    # populate the list
    for i in range(len(bucket_list)):
        this_str = str(bucket_list[i].key)
        if 'gz' in this_str:
            endme = this_str[-22:-4]
            fmt = '%Y%m%d_%H%M%S_V0'
            dt = datetime.strptime(endme, fmt)
            datetimes.append(dt)
            keys.append(bucket_list[i])

        if this_str[-3::] == 'V06':
            endme = this_str[-19::]
            fmt = '%Y%m%d_%H%M%S_V06'
            dt = datetime.strptime(endme, fmt)
            datetimes.append(dt)
            keys.append(bucket_list[i])

    
    # find the closest available radar to your datetime
    closest_datetime_b = _nearestDate(datetimes, datetime_t)
    closest_datetime_e = _nearestDate(datetimes, datetime_te)

    index_b = datetimes.index(closest_datetime_b)
    index_e = datetimes.index(closest_datetime_e)

    radar_namelist = keys[index_b:index_e+1]
    radar_list=[]
    for i in range(np.shape(radar_namelist)[0]):
        localfile = tempfile.NamedTemporaryFile()
        radar_namelist[i].get_contents_to_filename(localfile.name)
        radar_list.append(pyart.io.read(localfile.name))
    return radar_namelist,radar_list

def getLocation(lat1, lon1, brng, distancekm):
    lat1 = lat1 * np.pi / 180.0
    lon1 = lon1 * np.pi / 180.0
    #earth radius
    R = 6378.1
    #R = ~ 3959 MilesR = 3959
    bearing = (brng / 90.)* np.pi / 2.

    lat2 = np.arcsin(np.sin(lat1) * np.cos(distancekm/R) + np.cos(lat1) * np.sin(distancekm/R) * np.cos(bearing))
    lon2 = lon1 + np.arctan2(np.sin(bearing)*np.sin(distancekm/R)*np.cos(lat1),np.cos(distancekm/R)-np.sin(lat1)*np.sin(lat2))
    lon2 = 180.0 * lon2 / np.pi
    lat2 = 180.0 * lat2 / np.pi
    return lat2, lon2

def _nearestDate(dates, pivot):
    return min(dates, key=lambda x: abs(x - pivot))

def find_nearest(array, value):
    '''
    Function to find index of the array in which the value is closest to

    Parameters: array (array), value (number)
    Returns: index (int)

    Example: xind = CM1calc.find_nearest(x,5)
    '''

    array = np.asarray(array)
    idx = (np.abs(array-value)).argmin()
    return idx


def vehicle_correction_vad(radar,df):
    '''
    Function that creates a 'vad_corrected_velocity' field that can be used for vad calculations, 
    but should be general enough to use for stationary VADs as well as moving PPIs. 
    Other than adding the new field, the radar times are smoothly interpolated and the azimuths are 
    corrected via the GPS pandas dataframe.
    
    Parameters: pyart radar object (object), pandas dataframe of appropriate radarGPS file (dataframe)
    Returns: pyart radar object (object), speed (float), speed variance (float), bearing (float), bearing variance (float), 
             latitude (float), latitude variance (float), longitude (float), longitude variance (float)
    
    Example: radar, velmean, velvar, bearmean, bearvar, latmean, latvar, lonmean, lonvar = vehicle_correction_vad(radar,df)

    p.s. only works if the velocity is already dealiased and there is a 'corrected_velocity' field
         also only works if a single sweep is extracted, example: radar = radar.extract_sweeps([0])
    '''
    
    #orders the time to increase monotonically instead of having a massive step jump in the middle
    roll_mag = (np.argmax(np.abs(np.gradient(radar.time['data'])))+1)
    times = np.roll(radar.time['data'],-roll_mag) 
    
    #a complicated way to create linear increasing times (instead of steps) that start at 0 seconds after the time datum and increase to the middle of the second max time plateau (if confused, plotting it is helpful)
    #from now on, we are going to assume ray_times is the fractional seconds after the time datum the ray is gathered, and we need to roll it back to match with the rest of the data
    ray_times = np.roll(np.arange(0,((np.unique(times)[-2])/(find_nearest(times,np.unique(times)[-2])+int(np.sum(radar.time['data']==np.unique(times)[-2])/2)))*len(times)+1e-11,((np.unique(times)[-2])/(find_nearest(times,np.unique(times)[-2])+int(np.sum(radar.time['data']==np.unique(times)[-2])/2)))),roll_mag)

    radar.time['data']=ray_times


    df['datetime'] = [datetime.strptime(d,'%d%m%y%H%M%S') for d in df['ddmmyy']+[f'{h:06}' for h in df['hhmmss[UTC]'].astype(int)]]
    beginscanindex = df.loc[df['datetime'] == datetime.strptime(radar.time['units'],'seconds since %Y-%m-%dT%H:%M:%SZ')].index
    endscanindex = df.loc[df['datetime'] == datetime.strptime(radar.time['units'],'seconds since %Y-%m-%dT%H:%M:%SZ')].index+np.ceil(np.amax(ray_times))+1
    

    #else:
    
    dfscan = df.iloc[beginscanindex[0].astype(int):endscanindex[0].astype(int)]
    dfscan = dfscan.astype({'Bearing[degrees]': 'float'})
    dfscan = dfscan.astype({'Velocity[knots]': 'float'})
    
    ray_bearings = np.interp(ray_times,np.arange(len(dfscan)),dfscan['Bearing[degrees]'])
    ray_speeds = np.interp(ray_times,np.arange(len(dfscan)),dfscan['Velocity[knots]'])
    
    print('velocity [kts]',dfscan['Velocity[knots]'].mean(),'+-',dfscan['Velocity[knots]'].var())
    speed = dfscan['Velocity[knots]'].mean()
    print('bearing',dfscan['Bearing[degrees]'].mean(),'+-',dfscan['Bearing[degrees]'].var())
    bearing = dfscan['Bearing[degrees]'].mean()
    print('latitude',dfscan['Latitude'].astype(float).mean(),'+-',dfscan['Latitude'].astype(float).var())
    lat = dfscan['Latitude'].astype(float).mean()
    print('longitude',dfscan['Longitude'].astype(float).mean(),'+-',dfscan['Longitude'].astype(float).var())
    lon = dfscan['Longitude'].astype(float).mean()
    
    radar.azimuth['data'] += ray_bearings[:-1] #bearing
    
    rad_vel = copy.deepcopy(radar.fields['corrected_velocity'])
    
    rad_vel['data']+=(np.cos(np.deg2rad(radar.azimuth['data']-ray_bearings[:-1]))*(ray_speeds[:-1]/1.94384)*np.cos(np.deg2rad(radar.fixed_angle['data'][0])))[:,np.newaxis]
    
     #fix mask, remove points very close to radar as well as the very last bin, more often than not, = bad data
    rad_vel['data'].mask[:,:5] = True
    rad_vel['data'].mask[:,-1] = True
    radar.add_field('vad_corrected_velocity', rad_vel, replace_existing=True)
    
    return radar, dfscan['Velocity[knots]'].mean(),dfscan['Velocity[knots]'].var(),dfscan['Bearing[degrees]'].mean(),dfscan['Bearing[degrees]'].var(),dfscan['Latitude'].astype(float).mean(),dfscan['Latitude'].astype(float).var(),dfscan['Longitude'].astype(float).mean(),dfscan['Longitude'].astype(float).var()

In [ ]:
ka1_vads_df = pd.read_csv('/Users/juliabman/Desktop/vads/vad_dfs/ka1/05232022/ka1_vads_df')
ka2_vads_df = pd.read_csv('/Users/juliabman/Desktop/vads/vad_dfs/ka2/05232022/ka2_vads_df')
tobac_file = '/Users/juliabman/Desktop/research2024/tobac_Save/Track.nc'

In [ ]:
ka1_vads = sorted(glob.glob('/Users/juliabman/Desktop/vads/vads_radar_objects/ka1/05232022/*.nc'))
ka2_vads = sorted(glob.glob('/Users/juliabman/Desktop/vads/vads_radar_objects/ka2/05232022/*.nc'))

In [ ]:
ka1_vads_df

In [ ]:
ka2_vads_df

In [ ]:
#ka1_vads_df.drop(columns = 'Unnamed: 0')
#ka2_vads_df.drop(columns = 'Unnamed: 0')

In [ ]:
tobac_time_array_ka1 = np.array(ka1_vads_df.tobac_time)
tobac_time_array_datetime_ka1 = tobac_time_array_ka1.astype('datetime64[s]')

tobac_time_array_ka2 = np.array(ka2_vads_df.tobac_time)
tobac_time_array_datetime_ka2 = tobac_time_array_ka2.astype('datetime64[s]')

In [ ]:
tobac_features_xr = xr.open_dataset(tobac_file)
idx = tobac_features_xr['idx'].data
cell = tobac_features_xr['cell'].data
morton_storm_indeces_idx = np.where(idx == 29)
morton_storm_indeces = np.where(cell == 29)
print(morton_storm_indeces)
tobac_times = tobac_features_xr['time']
tobac_lats = np.array(tobac_features_xr['latitude'])
tobac_lons = np.array(tobac_features_xr['longitude'])

#morton_two_prior = morton_storm_indeces[405]
#morton_two_post = morton_storm_indeces[839]


morton_tobac_lats = tobac_lats[morton_storm_indeces]
morton_tobac_lons = tobac_lons[morton_storm_indeces]
morton_tobac_times = tobac_times[morton_storm_indeces]
morton_cell_idx_29 = cell[morton_storm_indeces]

#idx_29_lats = tobac_lats[morton_storm_indeces_idx]
#idx_29_lons = tobac_lons[morton_storm_indeces_idx]

morton_tobac_times_datetime = morton_tobac_times.astype('datetime64[s]')
print(morton_tobac_times_datetime)

# print(morton_tobac_lats)
# print(morton_tobac_lons)

In [ ]:
from math import sin, cos, sqrt, atan2, radians

def calc_velocity(lat1,lon1,lat2,lon2,time1,time2):
    R = 6371 # Radius of the earth in km
    dLat = radians(lat2-lat1)
    dLon = radians(lon2-lon1)
    rLat1 = radians(lat1)
    rLat2 = radians(lat2)
    a = sin(dLat/2) * sin(dLat/2) + cos(rLat1) * cos(rLat2) * sin(dLon/2) * sin(dLon/2) 
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    d = R * c * 1000 # Distance in m
    return d / (time2 - time1) # dividing by 1e9 because the times are in nanoseconds

def get_bearing(lat1, long1, lat2, long2):
    dLon = (long2 - long1)
    x = cos(radians(lat2)) * sin(radians(dLon))
    y = cos(radians(lat1)) * sin(radians(lat2)) - sin(radians(lat1)) * cos(radians(lat2)) * cos(radians(dLon))
    brng = np.arctan2(x,y)
    brng = (np.degrees(brng)+180) % 360
    return brng

In [ ]:
# FOR KA1
storm_velocity=[]
storm_direction=[]
#tobac_time_array_ka1 = np.array(ka1_vads_df.tobac_time)
time_initial = tobac_time_array_datetime_ka1[0].astype(float)
time_final = tobac_time_array_datetime_ka1[-1].astype(float)

for i in np.arange(0,19): # the range of the first tobac time index that is closest to the ka1 time and the last tobac time index closest to ka1 time
    velocity = calc_velocity(morton_tobac_lats[i], morton_tobac_lons[i],
                             morton_tobac_lats[i+1],morton_tobac_lons[i+1],
                             time_initial, time_final)
                             #storm_decimalsec[i],storm_decimalsec[i+1])
    storm_velocity.append(velocity)
    
    direction = get_bearing(morton_tobac_lats[i],morton_tobac_lons[i],
                            morton_tobac_lats[i+1], morton_tobac_lons[i+1])
    storm_direction.append(direction)
    
storm_velocity = np.append([np.nan],storm_velocity) # in meters/sec
storm_direction = np.append([np.nan],storm_direction) # in degrees

In [ ]:
storm_direction

In [ ]:
storm_directiondf = pd.DataFrame(storm_direction)

direction_corrected = storm_directiondf.dropna()

storm_directiondf= pd.DataFrame(storm_direction)

direction_corrected = storm_directiondf.dropna()

direction_corrected_array = np.array(direction_corrected)

In [ ]:
weights = np.array([0.05, 0.2, 0.5, 0.2, 0.05])
storm_direction_weighted_ka1 = []

for i in range(len((direction_corrected))):
    j = i+2 # if we start any further it'll loop to the back values of the array and average them
    #print(j)
    storm_direction_over_25_minutes = np.array([direction_corrected_array[j-2], direction_corrected_array[j-1], 
                                            direction_corrected_array[j], direction_corrected_array[j+1], direction_corrected_array[j+2]])
    #print(storm_direction_over_25_minutes)
    array = np.array(storm_direction_over_25_minutes)
    flat = array.flatten()
    #print(np.shape(flat))
    #print(np.shape(weights))
    #print(storm_direction_over_25_minutes)
    #storm_avg_weighted = sum(storm_direction_over_25_minutes * weights)/ sum(weights)
    storm_avg_weighted_ka1 = np.average(flat, weights = weights)
    print(storm_avg_weighted_ka1)
    storm_direction_weighted_ka1.append(storm_avg_weighted_ka1)

In [ ]:
for_now = ka1_vads_df[ka1_vads_df.tobac_time_indeces < 15]
for_now_index = for_now.tobac_time_indeces
for_now_array = np.array(for_now_index)

storm_direction_for_ka1_list = []
for t in for_now_array:
    #print(t)
    storm_direction_for_ka1 = storm_direction_weighted_ka1[t]
    storm_direction_for_ka1_list.append(storm_direction_for_ka1)

In [ ]:
# FOR KA2
storm_velocity_ka2=[]
storm_direction_ka2=[]

#tobac_time_array_ka2 = np.array(ka2_vads_df.tobac_time)
time_initial_ka2 = tobac_time_array_datetime_ka2[0].astype(float)
time_final_ka2 = tobac_time_array_datetime_ka2[-1].astype(float)

for i in np.arange(0,38): # the range of the first tobac time index that is closest to the ka1 time and the last tobac time index closest to ka1 time
    velocity_ka2 = calc_velocity(morton_tobac_lats[i], morton_tobac_lons[i],
                             morton_tobac_lats[i+1],morton_tobac_lons[i+1],
                             time_initial_ka2, time_final_ka2)
                             #storm_decimalsec[i],storm_decimalsec[i+1])
    storm_velocity_ka2.append(velocity_ka2)
    
    direction_ka2 = get_bearing(morton_tobac_lats[i],morton_tobac_lons[i],
                            morton_tobac_lats[i+1], morton_tobac_lons[i+1])
    storm_direction_ka2.append(direction_ka2)
    
storm_velocity_ka2 = np.append([np.nan],storm_velocity_ka2) # in meters/sec
storm_direction_ka2 = np.append([np.nan],storm_direction_ka2) # in degrees

storm_directiondf_ka2 = pd.DataFrame(storm_direction_ka2)

direction_corrected_ka2 = storm_directiondf_ka2.dropna()

storm_directiondf_ka2= pd.DataFrame(storm_direction_ka2)

direction_corrected_ka2 = storm_directiondf_ka2.dropna()

direction_corrected_array_ka2 = np.array(direction_corrected_ka2)

In [ ]:
weights = np.array([0.05, 0.2, 0.5, 0.2, 0.05])
storm_direction_weighted_ka2 = []

for i in range(len((direction_corrected_ka2))):
    j = i+2 # if we start any further it'll loop to the back values of the array and average them
    #print(j)
    storm_direction_over_25_minutes_ka2 = np.array([direction_corrected_array_ka2[j-2], direction_corrected_array_ka2[j-1], 
                                            direction_corrected_array_ka2[j], direction_corrected_array_ka2[j+1], direction_corrected_array_ka2[j+2]])
    #print(storm_direction_over_25_minutes)
    array_ka2 = np.array(storm_direction_over_25_minutes_ka2)
    flat_ka2 = array_ka2.flatten()
    #print(np.shape(flat))
    #print(np.shape(weights))
    #print(storm_direction_over_25_minutes)
    #storm_avg_weighted = sum(storm_direction_over_25_minutes * weights)/ sum(weights)
    storm_avg_weighted_ka2 = np.average(flat_ka2, weights = weights)
    print(storm_avg_weighted_ka2)
    storm_direction_weighted_ka2.append(storm_avg_weighted_ka2)

In [ ]:
for_now_ka2 = ka2_vads_df[ka2_vads_df.tobac_time_indeces < 34]
for_now_index_ka2 = for_now_ka2.tobac_time_indeces
print(for_now_ka2)
for_now_array_ka2 = np.array(for_now_index_ka2)

storm_direction_for_ka2_list = []
for t2 in for_now_array_ka2:
    #print(t)
    storm_direction_for_ka2 = storm_direction_weighted_ka2[t2]
    storm_direction_for_ka2_list.append(storm_direction_for_ka2)

In [ ]:
# FOR KA1
bearing_storm_ka1=[]
tobac_time_array_ka1 = np.array(ka1_vads_df.tobac_time)

for i in range(len(ka1_vads_df.tobac_lats)): # the range of the first tobac time index that is 
                            #closest to the ka1 time and the last tobac time index closest to ka1 time                            
    
    direction = get_bearing(ka1_vads_df.tobac_lats[i],ka1_vads_df.tobac_lons[i],
                            ka1_vads_df.Latmean[i], ka1_vads_df.Lonmean[i])
    bearing_storm_ka1.append(direction)
bearing_storm_ka1 = np.append([np.nan],bearing_storm_ka1) # in degrees

In [ ]:
bearing_df = pd.DataFrame(bearing_storm_ka1)

bearing_storm_ka1_corrected_df = bearing_df.dropna()

In [ ]:
bearing_storm_ka1_corrected = np.array(bearing_storm_ka1_corrected_df)

In [ ]:
storm_motion_correction = 90 -np.array(storm_direction_for_ka1_list)

bearing_storm_ka1_shifted_list = []
for thing in range(len(storm_motion_correction)):
    bearing_storm_ka1_shifted = bearing_storm_ka1_corrected[thing] + storm_motion_correction[thing]
    bearing_storm_ka1_shifted_list.append(bearing_storm_ka1_shifted)

In [ ]:
# FOR KA2
bearing_storm_ka2=[]
tobac_time_array_ka2 = np.array(ka2_vads_df.tobac_time)

for i_2 in range(len(ka2_vads_df.tobac_lats)): # the range of the first tobac time index that is 
                            #closest to the ka1 time and the last tobac time index closest to ka1 time                            
    
    direction2 = get_bearing(ka2_vads_df.tobac_lats[i_2],ka2_vads_df.tobac_lons[i_2],
                            ka2_vads_df.Latmean[i_2], ka2_vads_df.Lonmean[i_2])
    bearing_storm_ka2.append(direction2)
bearing_storm_ka2 = np.append([np.nan],bearing_storm_ka2) # in degrees


bearing_df_2 = pd.DataFrame(bearing_storm_ka2)
bearing_storm_ka2_corrected_df = bearing_df_2.dropna()
bearing_storm_ka2_corrected = np.array(bearing_storm_ka2_corrected_df)

In [ ]:
storm_motion_correction_ka2 = 90 - np.array(storm_direction_for_ka2_list)

bearing_storm_ka2_shifted_list = []
for thing_2 in range(len(storm_motion_correction_ka2)):
    bearing_storm_ka2_shifted = bearing_storm_ka2_corrected[thing_2] + storm_motion_correction_ka2[thing_2]
    bearing_storm_ka2_shifted_list.append(bearing_storm_ka2_shifted)

In [ ]:
# bring in the figure
distance_ka1_from_storm = ka1_vads_df.Distance_From_Storm
distance_ka2_from_storm = ka2_vads_df.Distance_From_Storm

# bring in the figure

fig = plt.figure(figsize = (5,5))
ax = fig.add_subplot(projection = 'polar')
ax.set_rmax(2)
ax.set_rlabel_position(-22.5)  # Move radial labels away from plotted line
ax.grid(True, linestyle = ':') # makes grid lines dotted
ax.set_title("Ka1 and Ka2 Scan Locations Adjusted by Storm Motion Using tobac")
ax.plot()

# now that we have the figure set up, we can plot on it
# r --> great circle distance from lat and lon of storm to lat and lon of vehicle
# theta --> corrected bearing
# in polar plots the order of plotting is (theta, r)

theta = np.array(bearing_storm_ka1_shifted_list)[22:] -90
theta_radians = theta * (np.pi/180)
r = distance_ka1_from_storm[22:120]# - np.array(ka1_matching_velocity) - np.array(storm_velocity)

#print(theta_radians)
#print(r)
print(np.shape(theta))
print(np.shape(r))

theta2 = np.array(bearing_storm_ka2_shifted_list) -90
theta2_radians = theta2 * (np.pi/180)
r2 = distance_ka2_from_storm[0:121]

print(np.shape(theta2_radians))
print(np.shape(r2))

ka1 = ax.scatter(theta_radians, r, s = 10) # s changes point size
ka2 = ax.scatter(theta2_radians, r2, s = 10)

ax.legend(['ka1', 'ka2'])
# plt.savefig('/Users/juliabman/Desktop/Seminar_Pictures/both_ka_positions.png')
plt.show()

Need to get the radar objects that align with the plotted points

In [ ]:
ka1_vads_read = []
for vad in ka1_vads:
    read = pyart.io.read(vad)
    ka1_vads_read.append(read)

ka2_vads_read = []
for vad in ka2_vads:
    read2 = pyart.io.read(vad)
    ka2_vads_read.append(read2)

In [ ]:
ka1_vads_df.Distance_From_Storm.values[22]

In [ ]:
r

In [ ]:
theta

In [ ]:
ka1_r = pd.Series(r)
ka1_theta=pd.Series(theta_radians.flatten())
ka1_radar = pd.Series(ka1_vads_read)
ka1_vads_filenames = pd.Series(ka1_vads)
ka1_df=pd.DataFrame(pd.concat([ka1_r, ka1_theta, ka1_radar, ka1_vads_filenames],axis=1))
ka1_df.columns=['r','theta','radar','filename']
print(np.shape(ka1_df))

#ka1_in_inflow=ka1_df[(ka1_df.r <= 40) & (ka1_df.theta <=90) or (ka1_df.theta > 270)]

ka1_negative_angles=ka1_df[(ka1_df.theta < 0)]
ka1_positive_angles=ka1_df[(ka1_df.theta >= 0)]
#print(ka1_negative_angles)

ka1_in_inflow_negative=ka1_negative_angles[(ka1_negative_angles.theta >= -(np.pi/2)) | (ka1_negative_angles.theta <=-(3 * np.pi)/2) & (ka1_negative_angles.r <=40)]
#ka1_in_inflow_positive = ka1_positive_angles[(ka1_positive_angles.theta <= (np.pi/2)) | (ka1_positive_angles.theta >= (3 * np.pi)/2) & (ka1_positive_angles.r <= 40)]

print(np.shape(ka1_in_inflow_negative))

In [ ]:
ka1_in_inflow_negative

In [ ]:
# ka1_r = pd.Series(r)
# ka1_theta=pd.Series(theta_radians.flatten())

# r_indeces = ka1_r.index.values
# ka1_radar = pd.Series(ka1_vads_read)
# ka1_vads_in_inflow = ka1_radar[r_indeces]
# ka1_vads_in_inflow_pd = pd.Series(ka1_vads_in_inflow)

# ka1_df=pd.DataFrame(pd.concat([ka1_r, ka1_theta, ka1_vads_in_inflow_pd],axis=1))
# ka1_df.columns=['r','theta','radar']
# print(np.shape(ka1_df))

# #ka1_in_inflow=ka1_df[(ka1_df.r <= 40) & (ka1_df.theta <=90) or (ka1_df.theta > 270)]

# ka1_negative_angles=ka1_df[(ka1_df.theta < 0)]
# ka1_positive_angles=ka1_df[(ka1_df.theta >= 0)]
# #print(ka1_negative_angles)

# ka1_in_inflow_negative=ka1_negative_angles[(ka1_negative_angles.theta >= -(np.pi/2)) | (ka1_negative_angles.theta <=-(3 * np.pi)/2) & (ka1_negative_angles.r <=40)]
# #ka1_in_inflow_positive = ka1_positive_angles[(ka1_positive_angles.theta <= (np.pi/2)) | (ka1_positive_angles.theta >= (3 * np.pi)/2) & (ka1_positive_angles.r <= 40)]

# print(np.shape(ka1_in_inflow_negative))

In [ ]:
ka1_in_inflow_negative = ka1_in_inflow_negative.dropna()

In [ ]:
ka1_in_inflow_negative

Check that the times of the vads align with when they were in the inflow or even plot the vads with the reflectivity thing alex made to check

In [ ]:
for t in ka1_in_inflow_negative.radar:
    fig = plt.figure()
    radar_map_display_r = pyart.graph.RadarMapDisplay(t)
    radar_map_display_r.plot_ppi_map('corrected_velocity_terminal', cmap = 'pyart_balance', vmin = -40, vmax = 40)
    plt.show()

In [ ]:
ka2_r = pd.Series(r2)
ka2_theta=pd.Series(theta2_radians.flatten())
ka2_radar = pd.Series(ka2_vads_df.Radar)
ka2_df=pd.DataFrame(pd.concat([ka2_r,ka2_theta, ka2_radar],axis=1))
ka2_df.columns=['r2','theta2','radar2']
print(np.shape(ka2_df))

#ka1_in_inflow=ka1_df[(ka1_df.r <= 40) & (ka1_df.theta <=90) or (ka1_df.theta > 270)]

ka2_negative_angles=ka2_df[(ka2_df.theta2 < 0)]
ka2_positive_angles=ka2_df[(ka2_df.theta2 >= 0)]
#print(ka1_negative_angles)

ka2_in_inflow_negative=ka2_negative_angles[(ka2_negative_angles.theta2 >= -(np.pi/2)) | (ka2_negative_angles.theta2 <=-(3 * np.pi)/2) & (ka2_negative_angles.r2 <=40)]
#ka1_in_inflow_positive = ka1_positive_angles[(ka1_positive_angles.theta <= (np.pi/2)) | (ka1_positive_angles.theta >= (3 * np.pi)/2) & (ka1_positive_angles.r <= 40)]

print(np.shape(ka2_in_inflow_negative))

In [ ]:
ka1_vads_df[42:]

In [ ]:
# bring in the figure
distance_ka1_from_storm = ka1_vads_df.Distance_From_Storm
distance_ka2_from_storm = ka2_vads_df.Distance_From_Storm

# bring in the figure

fig = plt.figure(figsize = (5,5))
ax = fig.add_subplot(projection = 'polar')
ax.set_rmax(2)
ax.set_rlabel_position(-22.5)  # Move radial labels away from plotted line
ax.grid(True, linestyle = ':') # makes grid lines dotted
ax.set_title("Ka1 and Ka2 Scan Locations Adjusted by Storm Motion Using tobac")
ax.plot()

# now that we have the figure set up, we can plot on it
# r --> great circle distance from lat and lon of storm to lat and lon of vehicle
# theta --> corrected bearing
# in polar plots the order of plotting is (theta, r)

# theta = np.array(bearing_storm_ka1_shifted_list)[22:] -90
# theta_radians = theta * (np.pi/180)
# r = distance_ka1_from_storm[22:120]# - np.array(ka1_matching_velocity) - np.array(storm_velocity)

# #print(theta_radians)
# #print(r)
# print(np.shape(theta))
# print(np.shape(r))

# theta2 = np.array(bearing_storm_ka2_shifted_list) -90
# theta2_radians = theta2 * (np.pi/180)
# r2 = distance_ka2_from_storm[0:121]

# print(np.shape(theta2_radians))
# print(np.shape(r2))

ka1 = ax.scatter(ka1_in_inflow_negative.theta, ka1_in_inflow_negative.r, s = 10) # s changes point size
ka2 = ax.scatter(ka2_in_inflow_negative.theta2, ka2_in_inflow_negative.r2, s = 10)

ax.legend(['ka1', 'ka2'])
# plt.savefig('/Users/juliabman/Desktop/Seminar_Pictures/both_ka_positions.png')
plt.show()

In [ ]:
for vad in ka1_vads[50:55]:
    read = pyart.io.read(vad)
    fig = plt.figure()
    rmd = pyart.graph.RadarMapDisplay(read)
    rmd.plot_ppi_map('corrected_velocity_terminal', cmap = 'pyart_balance', vmin = -40, vmax = 40)
    plt.show()

In [ ]:
SMALL_SIZE = 30
MEDIUM_SIZE = 40
BIGGER_SIZE = 60

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

#vad_files = test
#vadu_list = []
#vadv_list = []

#119 east
#170 south
#186 west
#191 north
for thefile in ka1_in_inflow_negative.filename:
    print(thefile)
    radar = pyart.io.read(thefile)
    if np.logical_and(radar.scan_type == 'ppi', radar.fixed_angle['data'][0] > 10):
        
        radar = radar.extract_sweeps([0])
        #radar = dealias_Ka(radar)
        
        df = pd.read_csv('/Users/juliabman/Desktop/research2024/GPS_Ka1_20220523.txt', dtype=str)
        
        radar, velmean, velvar, bearmean, bearvar, latmean, latvar, lonmean, lonvar = vehicle_correction_vad(radar,df)
        radar = pyart.io.read(thefile)

        try:
            VAD = pyart.retrieve.vad_browning(radar, 'corrected_velocity_terminal', z_want=np.arange(50,5001,10),gatefilter=None)
        except:
            print('not enough data')
            continue
        vadu = VAD.u_wind*1.94384 # meters to knots
        vadv = VAD.v_wind*1.94384
        print(f'u velocity {vadu[:10]}')
        print(f'v velocity {vadv[:10]}')
        
        fig = plt.figure(figsize=(51, 17),facecolor='white')
        display = pyart.graph.RadarDisplay(radar)
        
        ax1 = plt.subplot2grid((1,3),(0,0))
        display.plot_ppi('corrected_velocity_terminal', 0, cmap='pyart_balance', ax=ax1, vmin=-40., vmax=40.,
                             colorbar_label=r'Radial Velocity [m s$^{-1}$]') #title='Motion Corrected Radial Velocity')
        
        ax3 = plt.subplot2grid((1,3),(0,2))
        
        hodo = metpy.plots.Hodograph(ax3,component_range=40.)
        hodo.add_grid(increment=5)
        
        hodo.plot(vadu,vadv)

        ax3.set_xlabel('U-component Wind [knots]')
        ax3.set_ylabel('V-component Wind [knots]')
        ax3.set_title('Vehicle motion corrected VAD')

        # base = pyplot.gca().transData
        # rot = transforms.Affine2D().rotate_deg(90)
        
        for i,z in enumerate([0,95,195,295,395,495]):
            try:
                if np.isnan(vadu[z]):
                    pass
                else:
                    circle = plt.Circle((vadu[z], vadv[z]), radius=1.5,color='k',zorder=30)
                    ax3.add_patch(circle)
                    label = ax3.annotate(i, xy=(vadu[z], vadv[z]), fontsize=12, ha="center",va='center',color='white',zorder=30,weight="bold")
            except: pass
        
        
        r=35
        radar_namelist, radar_list = get_radar_from_aws_sam('KLBB', datetime.strptime(thefile.split('/')[-1].split('.')[0][29:],'%Y%m%d%H%M%S'), datetime.strptime(thefile.split('/')[-1].split('.')[0][29:],'%Y%m%d%H%M%S')+timedelta(minutes=5))
        print(radar_namelist)
        print(radar_list)
        #scantime88d = datetime.strptime(str(radar_namelist)[40:57], '%Y%m%d_%H%M%S') #SAM DELETED THAT
        swp_id = 0
        radar88d = radar_list[0].extract_sweeps([0])
        scantime88d = pyart.util.datetime_from_radar(radar88d)
        # SAM ADDED THAT^^^
        time_text = ' ' + str(radar88d.time['units'][-9:1])
        title ='KLBB PPI ' +scantime88d.strftime("%m/%d/%y %H:%M")+ ' UTC'
        display = pyart.graph.RadarMapDisplay(radar88d)
        ax2 = plt.subplot2grid((1,3),(0,1),projection=display.grid_projection)
        display.plot_ppi_map('reflectivity', swp_id, cmap='pyart_HomeyerRainbow', ax=ax2, vmin=0., vmax=75., 
                             colorbar_label='Radar Reflectivity [dBZ]', title='Reflectivity',
                             min_lon=getLocation(latmean, lonmean, 270, r)[1],
                             max_lon=getLocation(latmean, lonmean, 90, r)[1],
                             min_lat=getLocation(latmean, lonmean, 180, r)[0],
                             max_lat=getLocation(latmean, lonmean, 0, r)[0])
        
        m = matplotlib.markers.MarkerStyle(marker='^')
        m._transform = m.get_transform().rotate_deg(-bearmean)
        ax2.plot(lonmean,latmean,color='k',marker=m,ms=20,transform=ccrs.PlateCarree())
        m = matplotlib.markers.MarkerStyle(marker=3)
        m._transform = m.get_transform().rotate_deg(-bearmean)
        ax2.plot(lonmean,latmean,color='k',marker=m,ms=50,transform=ccrs.PlateCarree(),markeredgewidth=4)


        fname=thefile.split('/')[-1].split('.')[0][:]
        plt.suptitle(fname)#+' '+str(np.around(dfscan['Bearing[degrees]'].astype(float).var(),decimals=2))+' '+str(np.around(dfscan['Velocity[knots]'].astype(float).var(),decimals=2)))
        plt.tight_layout()
        # plt.savefig(f"/Users/juliabman/Desktop/alex_morton_hodos/{fname}.png")
        # plt.close()
        # fig.clf()
        # gc.collect()
        plt.show()

In [ ]:
SMALL_SIZE = 30
MEDIUM_SIZE = 40
BIGGER_SIZE = 60

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

#vad_files = test
#vadu_list = []
#vadv_list = []

#119 east
#170 south
#186 west
#191 north
for thefile in ka1_in_inflow_negative.filename:
    print(thefile)
    radar = pyart.io.read(thefile)
    if np.logical_and(radar.scan_type == 'ppi', radar.fixed_angle['data'][0] > 10):
        
        radar = radar.extract_sweeps([0])
        #radar = dealias_Ka(radar)
        
        df = pd.read_csv('/Users/juliabman/Desktop/research2024/GPS_Ka1_20220523.txt', dtype=str)
        
        radar, velmean, velvar, bearmean, bearvar, latmean, latvar, lonmean, lonvar = vehicle_correction_vad(radar,df)
        radar = pyart.io.read(thefile)

        try:
            VAD = pyart.retrieve.vad_browning(radar, 'velocity', z_want=np.arange(50,5001,10),gatefilter=None)
        except:
            print('not enough data')
            continue
        vadu = VAD.u_wind*1.94384 # meters to knots
        vadv = VAD.v_wind*1.94384
        print(f'u velocity {vadu[:10]}')
        print(f'v velocity {vadv[:10]}')
        
        fig = plt.figure(figsize=(51, 17),facecolor='white')
        display = pyart.graph.RadarDisplay(radar)
        
        ax1 = plt.subplot2grid((1,3),(0,0))
        display.plot_ppi('velocity', 0, cmap='pyart_balance', ax=ax1, vmin=-40., vmax=40.,
                             colorbar_label=r'Radial Velocity [m s$^{-1}$]') #title='Motion Corrected Radial Velocity')
        
        ax3 = plt.subplot2grid((1,3),(0,2))
        
        hodo = metpy.plots.Hodograph(ax3,component_range=40.)
        hodo.add_grid(increment=5)
        
        hodo.plot(vadu,vadv)

        ax3.set_xlabel('U-component Wind [knots]')
        ax3.set_ylabel('V-component Wind [knots]')
        ax3.set_title('Vehicle motion corrected VAD')

        # base = pyplot.gca().transData
        # rot = transforms.Affine2D().rotate_deg(90)
        
        for i,z in enumerate([0,95,195,295,395,495]):
            try:
                if np.isnan(vadu[z]):
                    pass
                else:
                    circle = plt.Circle((vadu[z], vadv[z]), radius=1.5,color='k',zorder=30)
                    ax3.add_patch(circle)
                    label = ax3.annotate(i, xy=(vadu[z], vadv[z]), fontsize=12, ha="center",va='center',color='white',zorder=30,weight="bold")
            except: pass
        
        
        r=35
        radar_namelist, radar_list = get_radar_from_aws_sam('KLBB', datetime.strptime(thefile.split('/')[-1].split('.')[0][29:],'%Y%m%d%H%M%S'), datetime.strptime(thefile.split('/')[-1].split('.')[0][29:],'%Y%m%d%H%M%S')+timedelta(minutes=5))
        print(radar_namelist)
        print(radar_list)
        #scantime88d = datetime.strptime(str(radar_namelist)[40:57], '%Y%m%d_%H%M%S') #SAM DELETED THAT
        swp_id = 0
        radar88d = radar_list[0].extract_sweeps([0])
        scantime88d = pyart.util.datetime_from_radar(radar88d)
        # SAM ADDED THAT^^^
        time_text = ' ' + str(radar88d.time['units'][-9:1])
        title ='KLBB PPI ' +scantime88d.strftime("%m/%d/%y %H:%M")+ ' UTC'
        display = pyart.graph.RadarMapDisplay(radar88d)
        ax2 = plt.subplot2grid((1,3),(0,1),projection=display.grid_projection)
        display.plot_ppi_map('reflectivity', swp_id, cmap='pyart_HomeyerRainbow', ax=ax2, vmin=0., vmax=75., 
                             colorbar_label='Radar Reflectivity [dBZ]', title='Reflectivity',
                             min_lon=getLocation(latmean, lonmean, 270, r)[1],
                             max_lon=getLocation(latmean, lonmean, 90, r)[1],
                             min_lat=getLocation(latmean, lonmean, 180, r)[0],
                             max_lat=getLocation(latmean, lonmean, 0, r)[0])
        
        m = matplotlib.markers.MarkerStyle(marker='^')
        m._transform = m.get_transform().rotate_deg(-bearmean)
        ax2.plot(lonmean,latmean,color='k',marker=m,ms=20,transform=ccrs.PlateCarree())
        m = matplotlib.markers.MarkerStyle(marker=3)
        m._transform = m.get_transform().rotate_deg(-bearmean)
        ax2.plot(lonmean,latmean,color='k',marker=m,ms=50,transform=ccrs.PlateCarree(),markeredgewidth=4)


        fname=thefile.split('/')[-1].split('.')[0][:]
        plt.suptitle(fname)#+' '+str(np.around(dfscan['Bearing[degrees]'].astype(float).var(),decimals=2))+' '+str(np.around(dfscan['Velocity[knots]'].astype(float).var(),decimals=2)))
        plt.tight_layout()
        # plt.savefig(f"/Users/juliabman/Desktop/alex_morton_hodos/{fname}.png")
        # plt.close()
        # fig.clf()
        # gc.collect()
        plt.show()